# Advanced Lane Finding

## Overview

This notebook contains an implementation to identify the lane boundaries in a video from a front-facing camera on a car. The camera calibration images, test road images, and project videos are located in this [repository](https://github.com/abmantha/CarND-Advanced-Lane-Lines.git)

* The goals / steps of this project are the following:
* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.
---

### Imports

In [1]:
import numpy as np
import cv2
import sklearn
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import imageio
imageio.plugins.ffmpeg.download()

from moviepy.editor import VideoFileClip
from IPython.display import HTML

### 1. Camera Calibration

__Finding Chessboard Corners__

In [2]:
# Create objpoints and imgpoints data from calibration images
%matplotlib qt
%matplotlib inline

images_files = glob.glob("camera_cal/calibration*.jpg")
nx = 9
ny = 6

def get_calibration_points(images): 
    objpoints = [] # Known object coordinates for chessboard
    imgpoints = []

    objp = np.zeros((ny*nx,3), np.float32) 
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)

#     figure, axis = plt.subplots(5, 4, figsize=(16, 16))
#     figure.subplots_adjust(hspace=0.4, wspace=0.3)
#     axis = axis.ravel()

    for index, fname in enumerate(images):
        # Read in a new image
        img = mpimg.imread(fname)
        
        # Convert image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)

        # If corners are found, add object points, image points
        if ret == True: 
            imgpoints.append(corners)
            objpoints.append(objp)
            img_with_chessboard = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)

#             axis[index].set_title(fname)
#             axis[index].axis('off')
#             axis[index].imshow(img_with_chessboard)
            
    return objpoints, imgpoints

objpoints, imgpoints = get_calibration_points(images_files)

### 2. Distortion Correction

#### Utility Functions

In [3]:
# Only need to apply this to a single image to retrieve calibration values
def get_camera_calibration_matrices(image, objpoints, imgpoints): 
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape, None, None)
    return mtx, dist

# Apply this to each image when undistortion
def undistort(image): 
    undistort = cv2.undistort(image, mtx, dist, None, mtx)
    return undistort

#### Undistortion of Calibration Images

In [ ]:
calibration_image = mpimg.imread(images_files[0])
mtx, dist = get_camera_calibration_matrices(calibration_image, objpoints, imgpoints)

figure, axis = plt.subplots(5, 4, figsize=(16, 16))
figure.subplots_adjust(hspace=0.4, wspace=0.3)
axis = axis.ravel()

for i, fname in enumerate(images_files): 
    img = mpimg.imread(fname)
    undistorted = undistort(img)
    axis[i].set_title(fname)
    axis[i].axis('off')
    axis[i].imshow(undistorted)

#### Undistortion of Test Images

In [ ]:
example_image_one = mpimg.imread("test_images/test1.jpg")
undistorted_example_image_one = undistort(example_image_one)
figure, (axis1, axis2) = plt.subplots(1, 2, figsize=(20, 15))
figure.subplots_adjust(hspace=0.4, wspace=0.3)
axis1.set_title("Original")
axis1.imshow(example_image_one)
axis2.set_title("Undistorted")
axis2.imshow(undistorted_example_image_one)

### 3. Perspective Transform

In [ ]:
def warp(image, src, dest): 
    # Get the transform matrix and inverse
    M = cv2.getPerspectiveTransform(src, dest)
    Minv = cv2.getPerspectiveTransform(dest, src)

    # Apply transform matrix to produce perspective-transformed image
    warped_image = cv2.warpPerspective(undistorted, M, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)

    return warped_image, M, Minv

#### Identify source and destination points on test images

In [ ]:
example_images = glob.glob("test_images/test*.jpg")
warped_example_images = []

for fname in example_images:
    example_image = mpimg.imread(fname)
    undistorted = undistort(example_image)
    image_size = undistorted.shape
    # define source and destination points for transform
    src = np.float32([(590,450),
                      (720,450), 
                      (200,680), 
                      (1050,680)])
    
    dest = np.float32([(400,0),
                  (800,0),
                  (400,image_size[0]),
                  (800,image_size[0])])

    warped_image, M, Minv = warp(example_image, src, dest)
    warped_example_images.append((warped_image, M, Minv))

    fig, axis = plt.subplots(1, 3, figsize=(20, 10))
    axis = axis.ravel()
    
    axis[0].set_title("Original")
    axis[0].imshow(example_image)

    axis[1].set_title("Undistorted Original")
    axis[1].imshow(undistorted)
    # plt.imshow(combined_thresholded_image, cmap='gray')
    axis[1].plot(590, 450, "o") # Top left
    axis[1].plot(720, 450, "o") # Top right
    axis[1].plot(200, 680, "o") # Bottom left
    axis[1].plot(1075, 680, "o") # Bottom right

    axis[2].set_title("Warped")
    axis[2].imshow(warped_image)
    axis[2].plot(0, 0, "o")
    axis[2].plot(example_image.shape[1], 0, "o")
    axis[2].plot(0, example_image.shape[0], "o")
    axis[2].plot(example_image.shape[1], example_image.shape[0], "o")

### 4. Color/Gradient Thresholding

In [4]:
def abs_sobel_threshold(image, orient='x', sobel_kernel=3, threshold=(0, 255)): 
    # 1) Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # 2) Take the derivative in x or y given orient='x' or 'y'
    xorder = 1 if orient == 'x' else 0
    sobel = cv2.Sobel(gray, cv2.CV_64F, xorder, not xorder, ksize=sobel_kernel)
    
    # 3) Take the absolute value of the derivative
    abs_sobel = np.absolute(sobel)
    
    # 4) Scale to 8-bit (0 - 255) then conver to type = np.uint8
    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    
    # 5) Create a mask of 1's where the scaled gradient magnitude is > threshold[0] and < threshold[1]
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= threshold[0]) & (scaled_sobel <= threshold[1])] = 1
    
    return binary_output

def magnitude_threshold(image, sobel_kernel=3, threshold=(0, 255)): 
    # 1) Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # 2) Take the derivative in x and y
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # 3) Calculate the magnitude
    sobel_x2 = np.square(sobel_x)
    sobel_y2 = np.square(sobel_y)
    abs_sobel_xy = np.sqrt(sobel_x2 + sobel_y2)
    
    # 4) Scale to 8-bit (0 - 255) then conver to type = np.uint8
    scaled_sobel = np.uint8(255 * abs_sobel_xy / np.max(abs_sobel_xy))
    
    # 5) Create a mask of 1's where the scaled gradient magnitude is > threshold[0] and < threshold[1]
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= threshold[0]) & (scaled_sobel <= threshold[1])] = 1
    
    return binary_output
 

def direction_threshold(image, sobel_kernel=3, threshold=(0, np.pi/2)): 
    # 1) Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # 2) Take the derivative in x and y
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # 3) Take the absolute x and y gradients
    abs_sobel_x = np.square(sobel_x)
    abs_sobel_y = np.square(sobel_y)
    
    # 4) Calculate the direction of gradient
    dir_sobel = np.arctan2(abs_sobel_y, abs_sobel_x)
    
    # 5) Create a mask of 1's where the scaled gradient magnitude is > threshold[0] and < threshold[1]
    binary_output = np.zeros_like(dir_sobel)
    binary_output[(dir_sobel >= threshold[0]) & (dir_sobel <= threshold[1])] = 1
    
    return binary_output

def binary_color_threshold(image, flag, c, threshold): 
    color_space = cv2.cvtColor(image, flag)
    channel = color_space[:, :, c]
    binary_color_space = np.zeros_like(channel)
    binary_color_space[(channel >= threshold[0]) & (channel <= threshold[1])] = 1
    return binary_color_space

### Testing threshold techniques

In [ ]:
warped_and_thresholded = []

for fname_raw in example_images: 
    raw_image = mpimg.imread(fname_raw)
    
    # Undistortion
    raw_image_undistorted = undistort(raw_image)
    img_shape = raw_image_undistorted.shape
    
    # Color thresholding
    # LAB
    lab_binary_threshold = binary_color_threshold(raw_image_undistorted, cv2.COLOR_RGB2Lab, 2, (145, 255))
    # LUV
    luv_binary_threshold = binary_color_threshold(raw_image_undistorted, cv2.COLOR_RGB2Luv, 0, (200, 255))
    # HLS
    hls_binary_threshold = binary_color_threshold(raw_image_undistorted, cv2.COLOR_RGB2HLS, 2, (200, 255))
    # HSV
    hsv_binary_threshold = binary_color_threshold(raw_image_undistorted, cv2.COLOR_RGB2HSV, 2, (180, 255))
    # RGB
    r = raw_image_undistorted[:, :, 0]
    r_binary = np.zeros_like(r)
    r_binary[(r >= 210) & (r <= 255)] = 1

    # For color thresholding combinations
    combined_color_binary = cv2.bitwise_or(lab_binary_threshold, luv_binary_threshold)

    # Gradient thresholding
    sobel_binary_x = abs_sobel_threshold(raw_image_undistorted, 'x', 3, (20, 100))
    sobel_binary_y = abs_sobel_threshold(raw_image_undistorted, 'y', 3, (20, 100))
    sobel_binary_magnitude = magnitude_threshold(raw_image_undistorted, 3, (20, 100))
    sobel_binary_direction = direction_threshold(raw_image_undistorted, 17, (0.7, 1.3))
    
    # For gradient thresholding combinations
    sobel_combined_1 = np.zeros_like(sobel_binary_x)
    sobel_combined_1[(sobel_binary_x == 1) | (sobel_binary_y == 1)] = 1
    sobel_combined_2 = np.zeros_like(sobel_binary_magnitude)
    sobel_combined_2[(sobel_binary_magnitude == 1) | (sobel_binary_direction == 1)] = 1
    
    # For color and gradient thresholding combinations
    combined_binary = np.zeros_like(sobel_binary_y)
    combined_binary[(sobel_binary_magnitude == 1) | (combined_color_binary == 1)] = 1
        
    # Source and destination points for transform
    src = np.float32([(580,450), (720,450), (200,680), (900,680)])
    dest = np.float32([(100,200), (800,200), (100,900), (800,900)])
    
    # Perspective transform
    raw_image_undistorted_warped, M, Minv = warp(combined_color_binary, src, dest)
    warped_and_thresholded.append(raw_image_undistorted_warped)
    
    # Plots
    f, axis = plt.subplots(1, 7, figsize=(20, 10))
    axis = axis.ravel()
    axis[0].set_title("Mask")
    axis[0].imshow(raw_image)
    
    axis[1].set_title("Undistorted")
    axis[1].imshow(raw_image_undistorted)
    
    axis[2].set_title("B Channel")
    axis[2].imshow(lab_binary_threshold, cmap='gray')
    
    axis[3].set_title("L Channel")
    axis[3].imshow(luv_binary_threshold, cmap='gray')
    
    axis[4].set_title("Thresholded")
    axis[4].imshow(combined_color_binary, cmap='gray')
    
    axis[5].set_title("Sobel")
    axis[5].imshow(sobel_binary_magnitude, cmap='gray')
    
    axis[6].set_title("Warped")
    axis[6].imshow(raw_image_undistorted_warped, cmap='gray')

### 5. Lane finding and Measuring Radius of Curvature

#### Lane Finding Method: Peaks in a Histogram & Sliding Window

In [5]:
def poly_fit_sliding_window(in_image): 
    # Take the histogram of the bottom half of the image
    histogram = np.sum(in_image[in_image.shape[0]//2:, :], axis=0)
    
    out_image = np.dstack((in_image, in_image, in_image)) * 255

    # Find the peak of the left ad right halves of the histogram
    # These are the starting points for left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    left_x_base = np.argmax(histogram[:midpoint])
    right_x_base = np.argmax(histogram[midpoint:midpoint+200]) + midpoint
    
    # Number of sliding windows
    n_sliding_windows = 9

    # Set the height of the windows
    window_height = np.int(in_image.shape[0] / n_sliding_windows)

    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = in_image.nonzero()
    nonzero_y = np.array(nonzero[0])
    nonzero_x = np.array(nonzero[1])

    # Current positions to be updated for each window
    left_x_current = left_x_base
    right_x_current = right_x_base

    # Width of windows +/- margin
    margin = 100

    # Minimum number of pixels found to recenter window
    minpix = 50

    # Create empty lists to receive left and right lane pixel indices
    left_lane_indices = []
    right_lane_indices = []

    # Step through windows one by one
    for window in range(n_sliding_windows): 
        # Identify window boundaries in x and y (for both left and right)
        window_y_low = in_image.shape[0] - (window + 1) * window_height
        window_y_high = in_image.shape[0] - window * window_height
        window_x_left_low = left_x_current - margin
        window_x_left_high = left_x_current + margin
        window_x_right_low = right_x_current - margin
        window_x_right_high = right_x_current + margin

        # Draw the windows on the visualization
        cv2.rectangle(out_image, (window_x_left_low, window_y_low), (window_x_left_high, window_y_high), (0, 255, 0), 2)
        cv2.rectangle(out_image, (window_x_right_low, window_y_low), (window_x_right_high, window_y_high), (0, 255, 0), 2)

        # Identify the nonzero pixels in x and y within the window
        good_left_indices = ((nonzero_y >= window_y_low) & (nonzero_y < window_y_high) & 
                             (nonzero_x >= window_x_left_low) & (nonzero_x < window_x_left_high)).nonzero()[0]
        good_right_indices = ((nonzero_y > window_y_low) & (nonzero_y < window_y_high) &
                             (nonzero_x >= window_x_right_low) & (nonzero_x < window_x_right_high)).nonzero()[0]

        # Append these indices to the lists
        left_lane_indices.append(good_left_indices)
        right_lane_indices.append(good_right_indices)

        # If you found > minpix pixels, recenter the next window on their mean position
        if len(good_left_indices) > minpix: 
            left_x_current = np.int(np.mean(nonzero_x[good_left_indices]))

        if len(good_right_indices) > minpix: 
            right_x_current = np.int(np.mean(nonzero_x[good_right_indices]))

    # Concatenate the arrays of indices
    left_lane_indices = np.concatenate(left_lane_indices)
    right_lane_indices = np.concatenate(right_lane_indices)

    # Extract left and right pixel positions
    left_x = nonzero_x[left_lane_indices]
    left_y = nonzero_y[left_lane_indices]
    right_x = nonzero_x[right_lane_indices]
    right_y = nonzero_y[right_lane_indices]

    # Fit a second order polynomial to each
    left_line_fit, right_line_fit = (None, None)
    if len(left_x) != 0: 
        left_line_fit = np.polyfit(left_y, left_x, 2)
    else: 
        print ("Left x DNE")
    
    if len(right_x) != 0:
        right_line_fit = np.polyfit(right_y, right_x, 2)
    else: 
        print ("Right x DNE")
        
    # return out_image, histogram, left_line_fit, right_line_fit, left_lane_indices, right_lane_indices, nonzero_x, nonzero_y 
    return left_line_fit, right_line_fit, left_lane_indices, right_lane_indices, (out_image, histogram)

#### Sliding Window Visualization

In [ ]:
for binary_warped in warped_and_thresholded:
    out_image, histogram, left_line_fit, right_line_fit, left_lane_indices, right_lane_indices, nonzero_x, nonzero_y = poly_fit_sliding_window(binary_warped)
    plot_y = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    left_fit_x = left_line_fit[0]*plot_y**2 + left_line_fit[1]*plot_y + left_line_fit[2]
    right_fit_x = right_line_fit[0]*plot_y**2 + right_line_fit[1]*plot_y + right_line_fit[2]

    out_image[nonzero_y[left_lane_indices], nonzero_x[left_lane_indices]] = [255, 0, 0]
    out_image[nonzero_y[right_lane_indices], nonzero_x[right_lane_indices]] = [0, 0, 255]
    
    f, axis = plt.subplots(1, 1, figsize=(10, 10))
    axis.imshow(out_image)
    axis.plot(left_fit_x, plot_y, color='y')
    axis.plot(right_fit_x, plot_y, color='y')
    axis.set_xlim(0, 1280)
    axis.set_ylim(720, 0)

#### Sliding Window Optimization for Next Frame

In [92]:
# Next frame represented by new binary_warped 
def poly_fit_on_next_frame(binary_warped, previous_left_line_fit, previous_right_line_fit):
    nonzero = binary_warped.nonzero()
    nonzero_y = np.array(nonzero[0])
    nonzero_x = np.array(nonzero[1])
    margin = 100

    left_lane_indices = ((nonzero_x > (previous_left_line_fit[0]*(nonzero_y**2) + previous_left_line_fit[1]*nonzero_y + 
                                       previous_left_line_fit[2] - margin)) & (nonzero_x < (previous_left_line_fit[0]*(nonzero_y**2) +
                                       previous_left_line_fit[1]*nonzero_y + previous_left_line_fit[2] + margin)))

    right_lane_indices = ((nonzero_x > (previous_right_line_fit[0]*(nonzero_y**2) + previous_right_line_fit[1]*nonzero_y + 
                                       previous_right_line_fit[2] - margin)) & (nonzero_x < (previous_right_line_fit[0]*(nonzero_y**2) +
                                       previous_right_line_fit[1]*nonzero_y + previous_right_line_fit[2] + margin)))

    # Extract left and right line pixel positions
    left_x = nonzero_x[left_lane_indices]
    left_y = nonzero_y[left_lane_indices]
    right_x = nonzero_x[right_lane_indices]
    right_y = nonzero_y[right_lane_indices]

    # Fit a second order polynomial to each
    left_line_fit = np.polyfit(left_y, left_x, 2)
    right_line_fit = np.polyfit(right_y, right_x, 2)
    
    return left_line_fit, right_line_fit, left_lane_indices, right_lane_indices

#### Example next frame visualization

In [93]:
for binary_warped in warped_and_thresholded:
    # Create an image to draw on and an image to show the selection window
    out_image = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_image = np.zeros_like(out_image)

    # Color in left and right line pixels
    out_image[nonzero_y[left_lane_indices], nonzero_x[left_lane_indices]] = (255, 0, 0)
    out_image[nonzero_y[right_lane_indices], nonzero_x[right_lane_indices]] = (0, 0, 255)

    # Generate a polygon to illustrate the search window area
    # Recast the x and y points into usable format for cv2.fillPoly()
    left_line_window_1 = np.array([np.transpose(np.vstack([left_fit_x - margin, plot_y]))])
    left_line_window_2 = np.array([np.flipud(np.transpose(np.vstack([left_fit_x + margin, plot_y])))])
    left_line_points = np.hstack((left_line_window_1, left_line_window_2))

    right_line_window_1 = np.array([np.transpose(np.vstack([right_fit_x - margin, plot_y]))])
    right_line_window_2 = np.array([np.flipud(np.transpose(np.vstack([right_fit_x + margin, plot_y])))])
    right_line_points = np.hstack((right_line_window_1, right_line_window_2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_image, np.int_([left_line_points]), (0, 255, 0))
    cv2.fillPoly(window_image, np.int_([right_line_points]), (0, 255, 0))
    result = cv2.addWeighted(out_image, 1, window_image, 0.3, 0)

    f, axis = plt.subplots(1, 1, figsize=(10, 10))
    axis.imshow(result)
    axis.plot(left_fit_x, plot_y, color='y')
    axis.plot(right_fit_x, plot_y, color='y')
    axis.set_xlim(0, 1280)
    axis.set_ylim(720, 0)

NameError: name 'warped_and_thresholded' is not defined

#### Measuring Radius of Curvature

In [124]:
def radius_of_curvature(binary_warped, left_lane_indices, right_lane_indices):
    plot_y = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    y_eval = np.max(plot_y)
    
    nonzero = binary_warped.nonzero()
    nonzero_y = np.array(nonzero[0])
    nonzero_x = np.array(nonzero[1])
        
    left_x = nonzero_x[left_lane_indices]
    left_y = nonzero_y[left_lane_indices]
    right_x = nonzero_x[right_lane_indices]
    right_y = nonzero_y[right_lane_indices]

    y_meters_per_pixel = 30 / 720
    x_meters_per_pixel = 3.7 / 700

    if len(left_x) != 0 and len(right_x) != 0: 
        # Fit new polynomials to x,y in world space
        left_line_fit_meters = np.polyfit(left_y * y_meters_per_pixel, left_x * x_meters_per_pixel, 2)
        right_line_fit_meters = np.polyfit(right_y * y_meters_per_pixel, right_x * x_meters_per_pixel, 2)

        # Calculate the new radii of curvature
        left_curve_rad = ((1 + (2*left_line_fit_meters[0]*y_eval*y_meters_per_pixel + left_line_fit_meters[1])**2)**1.5) / np.absolute(2*left_line_fit_meters[0])
        right_curve_rad = ((1 + (2*right_line_fit_meters[0]*y_eval*y_meters_per_pixel + right_line_fit_meters[1])**2)**1.5) / np.absolute(2*right_line_fit_meters[0])
    
        return left_curve_rad, right_curve_rad
    
    return (None, None)

def distance_from_center(binary_warped, left_line_fit, right_line_fit): 
    y_meters_per_pixel = 30 / 720
    x_meters_per_pixel = 3.7 / 700
    
    if left_line_fit is not None and right_line_fit is not None:         
#         plot_y = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
        y = binary_warped.shape[0]

        camera_center = binary_warped.shape[1] / 2
        left_fit_x = left_line_fit[0]*y**2 + left_line_fit[1]*y + left_line_fit[2]
        right_fit_x = right_line_fit[0]*y**2 + right_line_fit[1]*y + right_line_fit[2]

        lane_center = (left_fit_x + right_fit_x) / 2
        distance_from_center_in_meters = abs(camera_center - lane_center) * x_meters_per_pixel

        return distance_from_center_in_meters
    
    return None

#### Drawing the Lanes!

In [132]:
def draw_lanes(undistorted, binary_warped, Minv, left_line_fit, right_line_fit): 
    if left_line_fit is None or right_line_fit is None:
        return undistorted
    plot_y = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
#     print ("This is right before the error")
#     print ("Left line fit size: ", left_line_fit.shape)
#     print ("Right line fit size: ", right_line_fit.shape)
    left_fit_x = left_line_fit[0]*plot_y**2 + left_line_fit[1]*plot_y + left_line_fit[2]
#     print ("This is right after the error")
    right_fit_x = right_line_fit[0]*plot_y**2 + right_line_fit[1]*plot_y + right_line_fit[2]

    # Recast the x, y points into usable format for cv2.fillPoly()
    points_left = np.array([np.transpose(np.vstack([left_fit_x, plot_y]))])
    points_right = np.array([np.flipud(np.transpose(np.vstack([right_fit_x, plot_y])))])
    points = np.hstack((points_left, points_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([points]), (0, 255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    new_warp = cv2.warpPerspective(color_warp, Minv, (undistorted.shape[1], undistorted.shape[0]))

    # Combine the result with the original image
    result = cv2.addWeighted(undistorted, 1, new_warp, 0.3, 0)
    
    return result

### Pipeline Utilities

In [126]:
# Apply this to each image when undistortion
def undistort(image): 
    undistort = cv2.undistort(image, mtx, dist, None, mtx)
    return undistort

def threshold(image): 
    img_shape = image.shape
    
    # LAB
    lab_binary_threshold = binary_color_threshold(image, cv2.COLOR_RGB2Lab, 2, (145, 255))
    
    # LUV
    luv_binary_threshold = binary_color_threshold(image, cv2.COLOR_RGB2Luv, 0, (200, 255))

    # Combine color thresholds
    combined_color_binary = cv2.bitwise_or(lab_binary_threshold, luv_binary_threshold)
    
    # Gradient threshold
    sobel_binary_y = abs_sobel_threshold(image, 'y', 3, (20, 100))

    # Combine color and gradient thresholds
    combined_binary = np.zeros_like(sobel_binary_y)
    combined_binary[(sobel_binary_y == 1) | (combined_color_binary == 1)] = 1
    
    return combined_binary

def warped(image, src, dest): 
    # Get the transform matrix and inverse
    M = cv2.getPerspectiveTransform(src, dest)
    Minv = cv2.getPerspectiveTransform(dest, src)

    # Apply transform matrix to produce perspective-transformed image
    warped_image = cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)

    return warped_image, M, Minv

def process_image(image): 
    # 1. Undistort
    undistorted = undistort(image)
    
    # 2. Color/Gradient thresholding
    thresholded = threshold(image)
    
    # 3. Warp
    src = np.float32([(580,450), (720,450), (200,680), (900,680)])
    dest = np.float32([(100,200), (800,200), (100,900), (800,900)])
    
    warped_image, M, Minv = warped(thresholded, src, dest)
    
    return undistorted, thresholded, warped_image, M, Minv

# COME BACK TO FIX THIS AND MODIFY OR GIVE PROPER CREDIT TO JEREMY SHANNON
def draw_data(original_img, curv_rad, center_dist):
    new_img = np.copy(original_img)
    h = new_img.shape[0]
    font = cv2.FONT_HERSHEY_DUPLEX
    text = 'Curve radius: ' + '{:04.2f}'.format(curv_rad) + 'm'
    cv2.putText(new_img, text, (40,70), font, 1.5, (200,255,155), 2, cv2.LINE_AA)
    direction = ''
    if center_dist > 0:
        direction = 'right'
    elif center_dist < 0:
        direction = 'left'
    abs_center_dist = abs(center_dist)
    text = '{:04.3f}'.format(abs_center_dist) + 'm ' + direction + ' of center'
    cv2.putText(new_img, text, (40,120), font, 1.5, (200,255,155), 2, cv2.LINE_AA)
    return new_img

####  Line Class for Lane Tracking

In [127]:
# Stores characteristics of each line detection
# Create an instance for the left and right lanes
class Line(): 
    def __init__(self): 
        # was the line detected in the last iteration
        self.detected = False
        
        # x values of the last n fits of line
        self.recent_xfitted = [] 
        
        # average x values of the fitted line over the last n iterations
        self.bestx = None
        
        # polynomial coefficients averaged over last n iterations
        self.best_fit = None
        
        # polynomial coefficients for the most recent fit
        self.current_fit = []
        
        # radius of curvature of the line in some units
        self.radius_of_curvature = None
        
        # distance in meters of vehicle center from the line
        self.line_base_pos = None
        
        # difference in fit coefficients between last and new fits
        self.coefficient_diffs = np.array([0, 0, 0], dtype='float')
        
        self.px_count = None
        
    # COME BACK TO FIX THIS AND MODIFY
    def add_fit(self, fit, inds):
        # add a found fit to the line, up to n
        if fit is not None:
            if self.best_fit is not None:
                # if we have a best fit, see how this new fit compares
                self.diffs = abs(fit-self.best_fit)
            if (self.coefficient_diffs[0] > 0.001 or \
               self.coefficient_diffs[1] > 1.0 or \
               self.coefficient_diffs[2] > 100.) and \
               len(self.current_fit) > 0:
                # bad fit! abort! abort! ... well, unless there are no fits in the current_fit queue, then we'll take it
                self.detected = False
            else:
                self.detected = True
                self.px_count = np.count_nonzero(inds)
                self.current_fit.append(fit)
#                 print ("Appending current fit")
#                 print ("After appending new current fit: ", self.current_fit)
                if len(self.current_fit) > 5:
                    # throw out old fits, keep newest n
                    self.current_fit = self.current_fit[len(self.current_fit)-5:]
                self.best_fit = np.average(self.current_fit, axis=0)
        # or remove one from the history, if not found
        else:
            self.detected = False
            if len(self.current_fit) > 0:
#                 print ("Length of current fit is greater than 0")
                # throw out oldest fit
                self.current_fit = self.current_fit[:len(self.current_fit)-1]
#                 print (self.current_fit)
            if len(self.current_fit) > 0:
#                 print ("Length of current fit is still greater than 0 after trimming fat")
                # if there are still any fits in the queue, best_fit is their average
                self.best_fit = np.average(self.current_fit, axis=0)
#                 print (self.best_fit)

# Complete Pipeline

* Undistort image
* Apply color and gradient thresholding
* Apply perspective transform
* Identify lane lines using sliding window
* Measure radius of curvature
* Draw lane lines back onto original image
* Return updated image

In [128]:
# 1. Calibrate camera
calibration_images = glob.glob("camera_cal/calibration*.jpg")
objpoints, imgpoints = get_calibration_points(calibration_images)
calibration_image = mpimg.imread(calibration_images[0])

# 2. Get calibration matrices
mtx, dist = get_camera_calibration_matrices(calibration_image, objpoints, imgpoints)
    
def pipeline(image): 
    # 1. Process the image (undistort, thresholding, perspective transform)
    undistorted, thresholded, warped_image, M, Minv = process_image(image)
    
    # 2. Identify lane lines 
    if not left_line.detected and not right_line.detected: 
        # Use sliding window
        left_line_fit, right_line_fit, left_lane_indices, right_lane_indices, extra = poly_fit_sliding_window(warped_image)
    else: 
        left_line_fit, right_line_fit, left_lane_indices, right_lane_indices = poly_fit_on_next_frame(warped_image, left_line.best_fit, right_line.best_fit)
        
    # 3. Sanity-check the new lines
    if left_line_fit is not None and right_line_fit is not None:
        # We are defining lines in terms of f(y) because lanes are vertical
        # y = np.linspace(0, image.shape[0]-1, image.shape[0])
        y = image.shape[0]
        # Measure radius of curvature / line effectiveness
        left_fit_x = left_line_fit[0]*y**2 + left_line_fit[1]*y + left_line_fit[2]
        right_fit_x = right_line_fit[0]*y**2 + right_line_fit[1]*y + right_line_fit[2]
        diff = abs(left_fit_x - right_fit_x)
        if abs(diff - 500) > 300:
            left_line_fit = None
            right_line_fit = None
            
    # 4. Add the new lines assuming they are reasonable
    left_line.add_fit(left_line_fit, left_lane_indices)
    right_line.add_fit(right_line_fit, right_lane_indices)
            
    # 5. Draw lane lines back onto original image
    left_line_curvature, right_line_curvature = radius_of_curvature(warped_image, left_lane_indices, right_lane_indices)
    distance = distance_from_center(warped_image, left_line_fit, right_line_fit)
    
    # 5. Return updated image
    left_best_fit = left_line.best_fit
    right_best_fit = right_line.best_fit
    if left_line.best_fit is not None and right_line.best_fit is not None: 
        updated_image_with_lanes = draw_lanes(undistorted, warped_image, Minv, left_best_fit, right_best_fit)
        left_rad, right_rad = radius_of_curvature(warped_image, left_lane_indices, right_lane_indices)
        center_distance = distance_from_center(warped_image, left_best_fit, right_best_fit)
        updated_image_with_lanes_and_data = draw_data(updated_image_with_lanes, (left_rad + right_rad)/2, center_distance)
    else: 
        updated_image_with_lanes_and_data = image
        
    return updated_image_with_lanes_and_data

# Video Playback

### Project Video

In [129]:
# Create left and right Line() instances
left_line = Line()
right_line = Line()

project_video_outname = 'project_video_with_lanes.mp4'
project_video = VideoFileClip("project_video.mp4")
project_video_updated = project_video.fl_image(pipeline) # -- pipeline for single image
%time project_video_updated.write_videofile(project_video_outname, audio=False)

This is right after the error
[MoviePy] >>>> Building video project_video_with_lanes.mp4
[MoviePy] Writing video project_video_with_lanes.mp4



  0%|          | 1/1261 [00:00<02:51,  7.34it/s]

This is right after the error



  0%|          | 2/1261 [00:00<02:51,  7.36it/s]

This is right after the error


...

This is right after the error
This is right after the error



100%|█████████▉| 1259/1261 [03:20<00:00,  6.24it/s]

This is right after the error
This is right after the error



100%|█████████▉| 1260/1261 [03:20<00:00,  6.25it/s]


This is right after the error
[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_with_lanes.mp4 

CPU times: user 4min 56s, sys: 41.5 s, total: 5min 37s
Wall time: 3min 21s


In [131]:
HTML("""
<video width="480" height="275" controls>
    <source src="{0}">
</video>
""".format(project_video_outname))

### Challenge Video 1

In [137]:
left_line = Line()
right_line = Line()

challenge_video_one_outname = "test_videos/challenge_video_1_with_lanes.mp4"
challenge_video_one = VideoFileClip("test_videos/challenge_video_1.mp4")
challenge_video_one_updated = challenge_video_one.fl_image(pipeline) # -- pipeline for single image
%time challenge_video_one_updated.write_videofile(challenge_video_one_outname, audio=False)

[MoviePy] >>>> Building video test_videos/challenge_video_1_with_lanes.mp4
[MoviePy] Writing video test_videos/challenge_video_1_with_lanes.mp4



100%|██████████| 251/251 [00:40<00:00,  6.08it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos/challenge_video_1_with_lanes.mp4 

CPU times: user 58.2 s, sys: 7.68 s, total: 1min 5s
Wall time: 41.1 s


In [138]:
HTML("""
<video width="480" height="275" controls>
    <source src="{0}">
</video>
""".format(challenge_video_one_outname))

### Challenge Video 2

In [139]:
challenge_video_two_outname = 'test_videos/challenge_video_2_with_lanes.mp4'
challenge_video_two = VideoFileClip("test_videos/challenge_video_2.mp4")
challenge_video_two_updated = challenge_video_two.fl_image(pipeline) # -- pipeline for single image
%time challenge_video_two_updated.write_videofile(challenge_video_two_outname, audio=False)

[MoviePy] >>>> Building video test_videos/challenge_video_2_with_lanes.mp4
[MoviePy] Writing video test_videos/challenge_video_2_with_lanes.mp4


100%|██████████| 485/485 [01:17<00:00,  5.34it/s]     | 1/485 [00:00<01:12,  6.72it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos/challenge_video_2_with_lanes.mp4 

CPU times: user 1min 53s, sys: 13.9 s, total: 2min 6s
Wall time: 1min 18s


In [140]:
HTML("""
<video width="480" height="275" controls>
    <source src="{0}">
</video>
""".format(challenge_video_two_outname))

### Harder Challenge Video 

In [141]:
harder_challenge_video_outname = 'test_videos/harder_challenge_video_with_lanes.mp4'
harder_challenge_video = VideoFileClip("test_videos/harder_challenge_video.mp4")
harder_challenge_video_updated = harder_challenge_video.fl_image(pipeline) # -- pipeline for single image
%time harder_challenge_video_updated.write_videofile(harder_challenge_video_outname, audio=False)

[MoviePy] >>>> Building video test_videos/harder_challenge_video_with_lanes.mp4
[MoviePy] Writing video test_videos/harder_challenge_video_with_lanes.mp4


100%|█████████▉| 1199/1200 [04:05<00:00,  6.08it/s]    | 1/1200 [00:00<04:50,  4.13it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos/harder_challenge_video_with_lanes.mp4 

CPU times: user 5min 25s, sys: 44.9 s, total: 6min 10s
Wall time: 4min 6s


In [142]:
HTML("""
<video width="480" height="275" controls>
    <source src="{0}">
</video>
""".format(harder_challenge_video_outname))

# DUMP!!!!!!!!

In [ ]:
# def plot_gradient_thresholds(fname):
#     image = mpimg.imread(fname)
#     undistorted_example_image = undistort(image, mtx, dist)
#     threshold = (180, 255)
#     gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#     abs_sobel_x = abs_sobel_threshold(image, 'x', 3, (50, 100))
#     abs_sobel_y = abs_sobel_threshold(image, 'y', 3, (50, 100))
#     magnitude_sobel = magnitude_threshold(image, 9, (30, 100))
#     direction_sobel = direction_threshold(image, 17, (0.7, 1.3))
#     binary_gray = np.zeros_like(gray)
#     binary_gray[(gray > threshold[0]) & (gray <= threshold[1])] = 1
#     combined = np.zeros_like(direction_sobel)
#     combined[((abs_sobel_x == 1) & (abs_sobel_y == 1)) | ((magnitude_sobel == 1) & (direction_sobel == 1))] = 1

#     # Plotting thresholded images
#     f, axs = plt.subplots(3, 3, figsize=(20,10))
#     axs = axs.ravel()
    
#     axs[0].set_title(fname)
#     axs[0].imshow(image)

#     axs[1].set_title('Gradient -- x-oriented')
#     axs[1].imshow(abs_sobel_x, cmap='gray')

#     axs[2].set_title('Gradient -- y-oriented')
#     axs[2].imshow(abs_sobel_y, cmap='gray')

#     axs[3].set_title(fname)
#     axs[3].imshow(image)

#     axs[4].set_title('Magnitude Threshold')
#     axs[4].imshow(magnitude_sobel, cmap='gray')

#     axs[5].set_title('Direction Threshold')
#     axs[5].imshow(direction_sobel, cmap='gray')

#     axs[6].set_title(fname)
#     axs[6].imshow(image)
    
#     axs[7].set_title('Gray Thresholded')
#     axs[7].imshow(binary_gray, cmap='gray')

#     axs[8].set_title('Combined Magnitude and Direction Gradient')
#     axs[8].imshow(combined, cmap='gray')
    
# def plot_converted_color_space_thresholds(fname, flag): 
#     image = mpimg.imread(fname)
#     undistorted_example_image = undistort(image, mtx, dist)
    
#     threshold = (90, 255)
#     converted_color_space = cv2.cvtColor(image, flag)
#     channel_1 = converted_color_space[:, :, 0]
#     channel_2 = converted_color_space[:, :, 1]
#     channel_3 = converted_color_space[:, :, 2]
    
#     binary_1 = np.zeros_like(channel_1)
#     binary_2 = np.zeros_like(channel_2)
#     binary_3 = np.zeros_like(channel_3)
    
#     binary_1[(channel_1 > threshold[0]) & (channel_1 <= threshold[1])] = 1
#     binary_2[(channel_2 > threshold[0]) & (channel_2 <= threshold[1])] = 1
#     binary_3[(channel_3 > threshold[0]) & (channel_3 <= threshold[1])] = 1
    
#     f, axs = plt.subplots(2, 4, figsize=(20, 10))
#     axs = axs.ravel()
    
#     axs[0].set_title(fname)
#     axs[0].imshow(image)
    
#     axs[1].set_title('1')
#     axs[1].imshow(channel_1)
    
#     axs[2].set_title('2')
#     axs[2].imshow(channel_2)
    
#     axs[3].set_title('3')
#     axs[3].imshow(channel_3)
    
#     axs[4].set_title(fname)
#     axs[4].imshow(image)
    
#     axs[5].set_title('Channel 1 Threshold')
#     axs[5].imshow(binary_1, cmap='gray')
    
#     axs[6].set_title('Channel 2 Threshold')
#     axs[6].imshow(binary_2, cmap='gray')
    
#     axs[7].set_title('Channel 3 Threshold')
#     axs[7].imshow(binary_3, cmap='gray')
    
# def plot_combined_thresholds(image, orientation, sobel_threshold, sobel_kernel, flag, c, color_threshold):
#     undistorted_example_image = undistort(image)
    
#     if orientation == 'x' or orientation == 'y': 
#         sobel_binary = abs_sobel_threshold(undistorted_example_image, orientation, sobel_kernel, sobel_threshold)
#     elif orientation == 'm': 
#         sobel_binary = magnitude_threshold(undistorted_example_image, sobel_kernel, sobel_threshold)
#     elif orientation == 'd': 
#         sobel_binary = direction_threshold(undistorted_example_image, sobel_kernel, sobel_threshold)
#     elif orientation == "md": 
#         sobel_binary_magnitude = magnitude_threshold(undistorted_example_image, sobel_kernel, sobel_threshold)
#         sobel_binary_direction = magnitude_threshold(undistorted_example_image, sobel_kernel, sobel_threshold)        
        
#     # Threshold color channel
#     converted_color_space_image = cv2.cvtColor(undistorted_example_image, flag)
#     channel = converted_color_space_image[:, :, c]
#     channel_binary = np.zeros_like(channel)
#     channel_binary[(channel >= color_threshold[0]) & (channel <= color_threshold[1])] = 1

#     # Stack each channel to view their individual contributions in green and blue respectively
#     # This returns a stack of the two binary images, whose components you can see as different colors
#     color_binary = np.dstack(( np.zeros_like(sobel_binary), sobel_binary, channel_binary))

#     # Combine the two binary thresholds
#     combined_binary = np.zeros_like(sobel_binary)
#     combined_binary[(sobel_binary == 1) | (channel_binary == 1)] = 1

#     # Plotting thresholded images
#     f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
#     ax1.set_title('Stacked thresholds')
#     ax1.imshow(color_binary)

#     ax2.set_title('Gradient: ' + orientation + '--' + 'Color Space: ' + str(flag) + '--' + 'Color Channel: ' + str(c))
#     ax2.imshow(combined_binary, cmap='gray')
    
# def get_combined_thresholded_image(fname, orientation, sobel_threshold, sobel_kernel, flag, c, color_threshold):
#     image = mpimg.imread(fname)
#     undistorted_example_image = undistort(image)
    
#     if orientation == 'x' or orientation == 'y': 
#         sobel_binary = abs_sobel_threshold(undistorted_example_image, orientation, sobel_kernel, sobel_threshold)
#     elif orientation == 'm': 
#         sobel_binary = magnitude_threshold(undistorted_example_image, sobel_kernel, sobel_threshold)
#     elif orientation == 'd': 
#         sobel_binary = direction_threshold(undistorted_example_image, sobel_kernel, sobel_threshold)
#     elif orientation == "md": 
#         sobel_binary_magnitude = magnitude_threshold(undistorted_example_image, sobel_kernel, sobel_threshold)
#         sobel_binary_direction = magnitude_threshold(undistorted_example_image, sobel_kernel, sobel_threshold)        
        
#     # Threshold color channel
#     converted_color_space_image = cv2.cvtColor(undistorted_example_image, flag)
#     channel = converted_color_space_image[:, :, c]
#     channel_binary = np.zeros_like(channel)
#     channel_binary[(channel >= color_threshold[0]) & (channel <= color_threshold[1])] = 1

#     # Stack each channel to view their individual contributions in green and blue respectively
#     # This returns a stack of the two binary images, whose components you can see as different colors
#     color_binary = np.dstack(( np.zeros_like(sobel_binary), sobel_binary, channel_binary))

#     # Combine the two binary thresholds
#     combined_binary = np.zeros_like(sobel_binary)
#     combined_binary[(sobel_binary == 1) | (channel_binary == 1)] = 1

#     return combined_binary

# def plot_hsv_mask(image, y_threshold_min, y_threshold_max, w_threshold_min, w_threshold_max):
#     hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
#     yellow_lower_bound = np.array(y_threshold_min)
#     yellow_upper_bound = np.array(y_threshold_max)
#     white_lower_bound = np.array(w_threshold_min)
#     white_upper_bound = np.array(w_threshold_max)
    
#     yellow_mask = cv2.inRange(hsv, yellow_lower_bound, yellow_upper_bound)
#     white_mask = cv2.inRange(hsv, white_lower_bound, white_upper_bound)
# #     res = cv2.bitwise_or(yellow_mask, white_mask)
#     res = cv2.bitwise_and(image, image, mask=white_mask)
    
#         # Plotting thresholded images
#     f, axis = plt.subplots(1, 1, figsize=(10,10))
#     axis.set_title('Stacked thresholds')
#     axis.imshow(res)




# def region_of_interest(img, vertices):
#     """
#     Applies an image mask.
    
#     Only keeps the region of the image defined by the polygon
#     formed from `vertices`. The rest of the image is set to black.
#     """
#     #defining a blank mask to start with
#     mask = np.zeros_like(img)   
    
#     #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
#     if len(img.shape) > 2:
#         channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
#         ignore_mask_color = (255,) * channel_count
#     else:
#         ignore_mask_color = 255
        
#     #filling pixels inside the polygon defined by "vertices" with the fill color    
#     cv2.fillPoly(mask, vertices, ignore_mask_color)
    
#     #returning the image only where mask pixels are nonzero
#     masked_image = cv2.bitwise_and(img, mask)
#     return masked_image


    
# def color_mask(c_space, low, high):
#     mask = cv2.inRange(c_space, low, high)
#     return mask

# def apply_color_mask(image, mask):
#     res = cv2.bitwise_and(image, image, mask=mask)
#     return res

# def apply_color_mask2(hsv,img,low,high):
#     # Apply color mask to image
#     mask = cv2.inRange(hsv, low, high)
#     res = cv2.bitwise_and(img,img, mask= mask)
#     return res

# def apply_combined_mask(mask_1, mask_2): 
#     combined_mask = cv2.bitwise_xor(mask_1, mask_2)
#     return combined_mask